In [1]:
# Indusries
# Job Categories
# Locations ==> Suburb (optional)
# Job Title ==> Do (10) categories (cleaning) ==> Count Vectorizer
# Work Type
# Dates <== missing
# Job Description

# =============================================
#### Dummies
# =============================================
# ==> Industries
# ==> Location
# ==> Job Title

# =============================================
#### CountVectorizer
# =============================================
# ==> Job Title ==> Experiences columns (Senior, Junior, Entry-level)


# Step 1. Do Linear Regression

In [2]:
####
# Step 0: graph salary distribution (neg, pos skew)
# ==> EDA:
#     Contract DS (140 - 200k)
#     Contract DA (45 - 60k)
#     Contract (25%) | Fulltime (75%)
#     Industries (20-30) ==> to (10-15)
#     Job Titles ==> (DS [Junior, Senior/Manager, just DS], 
#                     DA [Junior, Senior/Manager, just DA]
#                     DE [Junior, Senior/Manager, just DE])
#                 ==> to (12-15)
#     Categories (200) ==> to (20)

# Step 1: Regression Problems
# ==> issues: 
#     not good model
# ==> treat as classification problem


In [3]:
import pandas as pd
import numpy as np
import re

# Folder
target_file = './training/training_next_16052019.csv'

In [4]:
frame = pd.read_csv(target_file, sep='|')
frame.head()

,content,industries,location,title,type_empl,url,wage
0,Over the last 2 years this business has been s...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Scientist Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/38892320?type=stan...,87575
1,The Data Architect is responsible for enterpri...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Architect Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/39015289?type=stan...,114181
2,About AFSA The Australian Financial Security A...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935314?type=stan...,87747
3,The Data Engineer (Cloud/Data Lake) will play ...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Engineer - Cloud / Data Lake Job in Brisb...,Full Time,https://www.seek.com.au/job/39015517?type=stan...,114181
4,About AFSA The Australian Financial Security A...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Applications Developer Enterprise Data Warehou...,Full Time,https://www.seek.com.au/job/38935330?type=stan...,87747


In [5]:
# Split Suburb out of content
def get_main_location(regex, sample):
#     print("In:", sample)
    result = re.search(regex, str(sample))

    if result:
#         print(result[0])
        result = result[0]
        result = result.replace(",","").lstrip().rstrip()
        return result
    elif len(str(sample)) > 3:
        return str(sample)
    return ''

# 
def get_suburb(regex, sample):
    result = re.sub(regex,"",str(sample))
    
    return result

In [6]:
frame['location'].value_counts()

Sydney, CBD, Inner West & Eastern Suburbs      511
Melbourne, CBD & Inner Suburbs                 200
Sydney                                          87
Brisbane, CBD & Inner Suburbs                   79
Sydney, North Shore & Northern Beaches          73
Sydney, Parramatta & Western Suburbs            72
Melbourne                                       40
Melbourne, Bayside & South Eastern Suburbs      38
Sydney, Ryde & Macquarie Park                   30
Brisbane                                        26
Melbourne, Western Suburbs                      22
Melbourne, Eastern Suburbs                      19
Sydney, South West & M5 Corridor                14
Melbourne, Northern Suburbs                     10
Sydney, Southern Suburbs & Sutherland Shire     10
Sydney, North West & Hills District              9
Brisbane, Northern Suburbs                       7
Brisbane, Southern Suburbs & Logan               5
Brisbane, Bayside & Eastern Suburbs              5
Brisbane, Western Suburbs & Ips

In [7]:
frame['suburb'] = frame['location'].map(lambda row: get_suburb(r"(^[A-Za-z]+, )",row))
frame['location'] = frame['location'].map(lambda row: get_main_location(r"(^[A-Za-z]+, )",row))

In [8]:
frame['location'].value_counts()

Sydney       806
Melbourne    329
Brisbane     126
Name: location, dtype: int64

In [9]:
frame['title'] = frame['title'].map(lambda row: row[:(row.find("Job")-1)])
frame['title'].value_counts()

Data Analyst                                                            15
Business Analyst                                                        14
Senior Business Analyst                                                 13
Test Analyst                                                             8
Financial Analyst                                                        8
Data Scientist                                                           7
Postdoctoral Research Fellow                                             7
Senior Data Scientist                                                    6
Campaign Analyst                                                         5
Senior Commercial Analyst                                                5
Campaign Project Officer                                                 5
Commercial Analyst                                                       5
Finance Analyst                                                          4
Manager Project Delivery 

In [12]:
fields = ['Data Analyst','Data Science','Data Scientist']
experiences = ['Senior','Sr','Jr','Junior','Manager','Supervisor','Lead']
developers = ['Engineer']

In [17]:
def get_exp(title):
    for exp in experiences:
        if exp in title:
            title = title.replace(exp,"")
            return title.lstrip().rstrip()
    return ''
    
frame['experience'] = frame['title'].map(get_exp)
frame['experience'].value_counts()

                                                                         867
Business Analyst                                                          16
Data Scientist                                                             8
Analyst                                                                    5
Commercial Analyst                                                         5
Advisor Human Resources                                                    4
Data Analyst                                                               4
Lecturer/ Lecturer in Cyber Security - Cryptography/Automation             4
Researcher and Team Leader, Information Security and Privacy               4
Product Manager | Mortgages                                                4
Project Delivery                                                           4
Product Manager                                                            3
APJ Marketing Analytics Manager                                            3